In [1]:
import matplotlib.pylab as pl
import matplotlib.animation as ani
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import glob
import json
import os

import cv2
import numpy as np
from skimage import color as skcl

from IPython.display import HTML

from common.data import colorize_image as CI

In [3]:
gpu_id = -1

load_size = 256
color_model = CI.ColorizeImageTorch(Xd = load_size)
color_model.prep_net(gpu_id, 'common/pretrained_models/checkpoints/siggraph_pretrained/latest_net_G.pth')

ColorizeImageTorch instantiated
path = common/pretrained_models/checkpoints/siggraph_pretrained/latest_net_G.pth
Model set! dist mode?  False
Using CPU for inference


In [4]:
def _unglob_files(files):
    files_found = []
    for file in files:
        globs_list = glob.glob(file)
        if not os.path.exists(file) and (len(globs_list) > 0):
            files_found.extend(globs_list)
        else:
            files_found.append(file)
    return files_found

In [5]:
INPUT_VIDEOS = sorted(_unglob_files([os.path.join('../../samples', '*.avi')]))
INPUT_FILE_NAME = INPUT_VIDEOS[2] # road.avi

INPUT_TASK_FILE = '../../samples/ui_road.json'

INPUT_FRAMES_MAX = 2

In [6]:
print(INPUT_FILE_NAME)
print(INPUT_TASK_FILE)

../../samples/road.avi
../../samples/ui_road.json


In [7]:
cap = cv2.VideoCapture(INPUT_FILE_NAME)

# First frame
_, sample_frame = cap.read()
frame_shape = sample_frame.shape
del sample_frame

# Optical flow params
lk_params = dict(
    winSize = (15, 15), maxLevel = 2, criteria = (cv2.TermCriteria_EPS | cv2.TermCriteria_COUNT, 10, 0.03)
)

In [8]:
task_list = []
with open(INPUT_TASK_FILE, 'r') as f:
    task_list = json.loads(f.read())

In [9]:
cap = cv2.VideoCapture(INPUT_FILE_NAME)

# First frame
old_frame = None

p0 = None
old_gray = None
p1 = np.array([[d['point']] for d in task_list]).astype(np.float32)
color = np.array([d['color'] for d in task_list]).astype(np.uint8)

images = []
track_frames = []

st = (p1[:, 0, :] > 1e-16).all(axis = 1).astype(np.uint8)
tracking_mask = np.zeros(frame_shape, dtype = np.uint8)

w = int(max((frame_shape[0], frame_shape[1])) / load_size)

for i in range(INPUT_FRAMES_MAX):
    _, frame = cap.read()
    if frame is None:
        break
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_l = skcl.rgb2lab(frame)[:, :, 0]
    frame_gray3 = cv2.resize(
        cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2RGB),
        (load_size, load_size),
        interpolation = cv2.INTER_CUBIC
    )
    
    im = np.zeros(frame_gray3.shape, dtype = np.uint8)
    mask = np.zeros(tuple(list(frame_gray3.shape[:-1]) + [1]), dtype = np.uint8)
    
    if p0 is not None:
        p1, st_new, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        st &= st_new.flatten()
        # Necessary to avoid overflow during mask preparation
        st &= (p1[:, 0, :] >= 1e-16).all(axis = 1).astype(np.uint8)
    
    for j in range(color.shape[0]):
        if st[j] == 0:
            continue
        
        point = ((p1[j, 0, :] * load_size) / np.array(frame_shape[1::-1])).astype(np.uint)
        
        if (point < w).any():
            st[j] = 0
            continue
        
        if p0 is not None:
            a, b = p0[j, 0, :].ravel()
            c, d = p1[j, 0, :].ravel()
            tracking_mask = cv2.line(tracking_mask, (a, b), (c, d), color[j].tolist(), 2)
        
        tl = tuple((point - w).tolist())
        br = tuple((point + w).tolist())
        
        cv2.rectangle(mask, tl, br, 255, -1)
        cv2.rectangle(im, tl, br, color[j].tolist(), -1)
    
    im_mask0 = (mask > 0.0).transpose((2, 0, 1))
    im_ab0 = skcl.rgb2lab(im).transpose((2, 0, 1))[1:3, :, :]
    
    color_model.set_image(frame_gray3)
    color_model.net_forward(im_ab0, im_mask0)
    
    out_ab = color_model.output_ab.transpose((1, 2, 0))
    out_ab = cv2.resize(out_ab, frame_shape[-2::-1], interpolation = cv2.INTER_CUBIC)
    out_lab = np.concatenate((frame_l[..., np.newaxis], out_ab), axis = 2)
    out_img = (np.clip(skcl.lab2rgb(out_lab), 0, 1) * 255).astype(np.uint8)
    
    images.append(out_img.copy())
    track_frames.append(tracking_mask.copy())
    
    old_gray = frame_gray.copy()
    p0 = p1

/home/learner/anaconda3/lib/python3.6/site-packages/skimage/color/colorconv.py:993: UserWarning: Color data out of range: Z < 0 in 113 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/home/learner/anaconda3/lib/python3.6/site-packages/skimage/color/colorconv.py:993: UserWarning: Color data out of range: Z < 0 in 53 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)


In [10]:
SHOW_TRACKS = True

In [11]:
ims = []
fig = pl.figure()

for i in range(len(images)):
    if SHOW_TRACKS:
        img = cv2.add(images[i], track_frames[i])
    else:
        img = images[i]
    imgs = pl.imshow(img, animated = True)
    ims.append([imgs,])

anim = ani.ArtistAnimation(fig, ims, interval = 100, blit = True, repeat_delay = 1000)
pl.close()

try:
    anim_html = anim.to_html5_video()
except Exception:
    anim_html = anim.to_jshtml()

# Delete the rogue temp-file
try:
    os.remove('None0000000.png')
except FileNotFoundError:
    pass

HTML(anim_html)

In [12]:
output_folder_base = os.path.dirname(os.path.abspath(INPUT_FILE_NAME))
output_folder_name = (os.path.split(os.path.abspath(INPUT_FILE_NAME))[-1]).split(os.path.extsep)[0] + '_out'

output_folder_path = os.path.join(output_folder_base, output_folder_name)
if not os.path.isdir(output_folder_path):
    os.mkdir(output_folder_path)

for i in range(len(images)):
    if SHOW_TRACKS:
        img = cv2.add(images[i], track_frames[i])
    else:
        img = images[i]
        
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    output_file_path = os.path.join(output_folder_path, (output_folder_name + '_%06d.png') % (i + 1))
    cv2.imwrite(output_file_path, img)